In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
# 桌面地址
local_path='C:/Users/wejia43/OneDrive - IKEA/Desktop/'

# 文本列名
name='工单内容'
# 金额列名
value="补偿金额"
# 唯一索引
auto="工单编号"

# 原始文件名
file_name='9月'
# 输出文件名
final_name='1233'

In [3]:
# 读取数据
data=pd.read_excel(local_path+file_name+".xlsx",sheet_name="Sheet1",index=False)

In [4]:
for i in range(0,data.shape[0]):
    # 去掉. -
    tmp=data.loc[i,name].replace(r'.','').replace(r'-','')
    # 查找数字
    num=re.findall(r'\d+',tmp)
    # 只保留长度为8的
    article=str([j for j in num if len(j)==8])
    # 去掉'
    article=article[1:len(article)-1].replace('\'','')
    # 生成新的一列用于存储
    data.loc[i,'Article_No']=article
    # 拆分备用列
    data.loc[i,'拆分备用']=article

In [13]:
# 获取列名
pd_cols=list(data.columns)
# 获取固定列索引
index_1=pd_cols[0:len(pd_cols)-1]
# 获取操作列索引
index_2=pd_cols[len(pd_cols)-1]
# 拆分货号为多行
df=data.set_index(index_1)[index_2].str.split(",",expand=True).stack().reset_index(drop=True,level=-1).reset_index().rename(columns={0: "Article_No_split"})


df['Article_No_split']=df['Article_No_split'].str.strip()

In [26]:
def maturity_count(data_col, val):
    """
    data_col : 输入data的某一列 如要输入pid这一列,则输入data.pid
    val : 特定data_col这一列索要查询的value值

    return  : 返回指定列，某一个值(val)出现多少次
    """

    data_pid_dict = data_col.value_counts()
    count = data_pid_dict[val]

    return count

In [ ]:
# 平均拆分金额
for j in range(0,df.shape[0]):
    tmp=df.loc[j,value]/maturity_count(df[auto],df.loc[j,auto])
    df.loc[j,'补偿金额_split']=tmp
df

In [14]:
# 输出
df.to_excel(local_path+final_name+".xlsx",index=False)